make sure both processes get same audio


-choose file
- process via diarizer

In [1]:
from pathlib import Path
import random

def ls(self):
    """
    List the contents of the directory.
    Returns:
        List[Path]: A list of Paths in the directory.
    """
    if not self.is_dir():
        raise NotADirectoryError(f"{self} is not a directory")
    
    return list(self.iterdir())
Path.ls = ls
from app.services.audio.audio import AudioSlicer
import json
import pandas as pd
import numpy as np

In [4]:
import random

In [5]:
audio_path = Path('/home/dima/ssd/1_feature/audio')

In [9]:
path = audio_path/'37e28028-1e32-4c5c-b356-9ce9623d2420.webm'

In [24]:
path = random.choice(audio_path.ls())

In [33]:


path = audio_path/'f2cb51cc-ccd1-4721-9701-f06b90b18e83.webm'

In [34]:
with open (path,'rb') as f:
    r = f.read()[:50]

In [35]:
r

b'\x1aE\xdf\xa3\xef\xbf\xbdB\xef\xbf\xbd\xef\xbf\xbd\x01B\xef\xbf\xbd\xef\xbf\xbd\x01B\xef\xbf\xbd\x04B\xef\xbf\xbd\x08B\xef\xbf\xbd\xef\xbf\xbdwebmB\xef\xbf\xbd\xef\xbf'

In [29]:
#path = audio_path/'test.webm'
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, 600)
audio_slicer.audio

In [3]:
def prep_transcripts(transcription):
    df = pd.DataFrame(transcription)
    df = df[[2,3,4]]
    df.columns = ['start','end','speech']
    df['start'] = pd.to_timedelta(df['start'],unit='s') + pd.Timestamp(seek)
    df['end'] = pd.to_timedelta(df['end'],unit='s') + pd.Timestamp(seek)
    
    return df

In [4]:
def prep_diarization(df):

    df['start'] = pd.to_timedelta(df['start'],unit='s')+pd.Timestamp(seek)
    df['end']   = pd.to_timedelta(df['end'],unit='s')+pd.Timestamp(seek)
    return df


In [5]:
def apply_diarization(trans_df,diar_df):
    segments = trans_df.to_dict('records')

    for seg in segments:
        diar_df['intersection'] = np.minimum(diar_df['end'], seg['end']) - np.maximum(diar_df['start'], seg['start'])
        speaker_ = diar_df[(diar_df['intersection'] == diar_df['intersection'].max())&(diar_df['intersection']>pd.Timedelta(0))]['speaker']
        if len(speaker_)>0:
            seg['speaker'] = speaker_.iloc[0]
            
    return pd.DataFrame(segments)

In [6]:
def split_to_multiple(df,step):
    df['duration'] = (df['end'] - df['start']).dt.total_seconds()
    df['group'] = (df['duration'].cumsum() // step).astype(int)
    return  [group.drop(columns = ['group']) for _, group in df.groupby('group')]

In [167]:
with open('2505_test_transcribe.json','r') as f:
    transcription = f.read()
with open('2505_test_diarize.json','r') as f:
    diarization_ = f.read()
transcription = json.loads(transcription)
diarization = pd.read_json(diarization_)
seek = pd.Timestamp.utcnow()
transcription_df = prep_transcripts(transcription)
diarization_df = prep_diarization(diarization)
transcription_step = 40
transcription_overlap = 2

diarization_step = 30
diarization_overlap = 10

last_mutable = pd.DataFrame(columns = ['start','end','speech','speaker'])
immutable = pd.DataFrame(columns = ['start','end'])

In [200]:
first_timestamp = transcription_df['start'].min();first_timestamp
trans_df = transcription_df[transcription_df['end']<first_timestamp+pd.Timedelta(seconds=transcription_step)]
diar_df = diarization_df[diarization_df['end']<first_timestamp+pd.Timedelta(seconds=diarization_step)]
# simulating pop bihavour
transcription_df = pd.concat([transcription_df,trans_df]).drop_duplicates(subset = ['start','end'],keep=False)
diarization_df = pd.concat([diarization_df,diar_df]).drop_duplicates(subset = ['start','end'],keep=False)

trans_df = pd.concat([trans_df,last_mutable[last_mutable['speaker'].isna()]])
mutable = apply_diarization(trans_df,diar_df)
if len(mutable)>0:
    rename_dict = {s:n for n,s in enumerate(mutable['speaker'].dropna().unique().tolist())}
    mutable = mutable.replace(rename_dict).sort_values('start')
    immutable = pd.concat([immutable,last_mutable.dropna()]).reset_index(drop = True)
    last_mutable= mutable.copy()

        

In [203]:
immutable.drop_duplicates()

,start,end,speech,speaker
0,2024-05-25 12:28:32.466065+00:00,2024-05-25 12:28:40.126065+00:00,У нас появился дизайнер для верстки презентац...,0.0
1,2024-05-25 12:28:40.506065+00:00,2024-05-25 12:28:46.766065+00:00,"Аня, я думаю, чуть позже мы с тобой сегодня у...",0.0
2,2024-05-25 12:28:47.826065+00:00,2024-05-25 12:28:49.446065+00:00,"Настя, а ты хочешь с нами в чат?",0.0
3,2024-05-25 12:28:49.746065+00:00,2024-05-25 12:28:50.986065+00:00,"Да, давайте, конечно.",1.0
4,2024-05-25 12:28:51.286065+00:00,2024-05-25 12:28:51.506065+00:00,Привет.,1.0
...,...,...,...,...
192,2024-05-25 12:38:08.766065+00:00,2024-05-25 12:38:09.266065+00:00,Ну да.,0.0
193,2024-05-25 12:38:09.406065+00:00,2024-05-25 12:38:11.566065+00:00,Вдруг они могут внутренний конкурс сделать.,0.0
194,2024-05-25 12:38:11.686065+00:00,2024-05-25 12:38:13.106065+00:00,Мы готовы и креатив помочь.,0.0
195,2024-05-25 12:38:13.766065+00:00,2024-05-25 12:38:14.906065+00:00,"Да, задача прикольная.",1.0


In [202]:
mutable

,start,end,speech,speaker
0,2024-05-25 12:38:16.786065+00:00,2024-05-25 12:38:17.546065+00:00,"Да, ладно.",NaN
1,2024-05-25 12:38:17.666065+00:00,2024-05-25 12:38:19.286065+00:00,"Вик, тогда пока не убирай.",NaN
2,2024-05-25 12:38:19.326065+00:00,2024-05-25 12:38:20.026065+00:00,Пусть висит.,NaN
3,2024-05-25 12:38:21.246065+00:00,2024-05-25 12:38:22.166065+00:00,Спасибо.,NaN
